In [ ]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *

#Downloading dataset
zipurl = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip'    
r = requests.get(zipurl, stream = True) 
  
with open("dataset.zip","wb") as pdf: 
    for chunk in r.iter_content(chunk_size=1024): 
         if chunk: 
            pdf.write(chunk)
            
# Extract all the contents of zip file in current directory            
with ZipFile('dataset.zip', 'r') as zipObj:
    zipObj.extractall()

In [5]:
#!/usr/bin/env python3
import os
import logging
from pathlib import Path
import requests 
from glob import glob
from zipfile import ZipFile

logging.basicConfig(level=logging.DEBUG)

#Import Pegasus API
from Pegasus.api import *


#Replica Catalog
rc = ReplicaCatalog()

in_files = list()
def get_files(d: Path) -> None:
    for p in d.iterdir():
        if p.is_file():
            f = File(p.name)
            in_files.append(f)
            rc.add_replica("local", p.name, str(p.resolve()))
            
            print("Added lfn: {}, pfn: {}".format(
                    p.name,
                    p.resolve()
                ))
        else:
            get_files(p)

get_files(Path("images"))

rc.write()



#Transformation
pre_process_resize = Transformation( "preprocess1.py",
        site = "local",
        pfn = "/home/scitech/shared-data/CatsAndDogs/preprocess1.py",
        is_stageable=True)

pre_process_augment = Transformation( "Augmentation.py",
        site = "local",
        pfn = str(Path(".").resolve() / "Augmentation.py"),
        is_stageable=True)

                                    
tc = TransformationCatalog()\
    .add_transformations(pre_process_resize,pre_process_augment)\
    .write()

#Workflow
wf = Workflow("Cats_and_Dogs", infer_dependencies=True)


resized_images = File('resized_images.txt')     
labels = File('labels.txt')

job_preprocess1 = Job(pre_process_resize)\
                    .add_inputs(*in_files)\
                    .add_outputs(labels,resized_images) 



aug_images_txt = File('augmentation.txt')
aug_labels_txt = File('aug_labels.txt')

job_preprocess2 = Job(pre_process_augment)\
                    .add_inputs(labels,resized_images)\
                    .add_outputs(aug_images_txt,aug_labels_txt)

wf.add_jobs(job_preprocess1,job_preprocess2)                                    

FileNotFoundError: [Errno 2] No such file or directory: 'images'

In [4]:
try:
     wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.29 15:43:27.479 UTC:
2020.07.29 15:43:27.485 UTC:   -----------------------------------------------------------------------
2020.07.29 15:43:27.491 UTC:   File for submitting this DAG to HTCondor           : Cats_and_Dogs-0.dag.condor.sub
2020.07.29 15:43:27.496 UTC:   Log of DAGMan debugging messages                 : Cats_and_Dogs-0.dag.dagman.out
2020.07.29 15:43:27.502 UTC:   Log of 

STAT  IN_STATE  JOB                                                                                            
Idle     00:00  Cats_and_Dogs-0 ( /home/scitech/shared-data/CatsAndDogs/scitech/pegasus/Cats_and_Dogs/run0021 )
Summary: 1 Condor job total (I:1)




In [ ]:
pwd